In [1]:
!pip install carla==0.9.12

In [2]:
import carla
import math
import random
import time

In [3]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [4]:
print(client.get_available_maps())

['/Game/Carla/Maps/Town01', '/Game/Carla/Maps/Town01_Opt', '/Game/Carla/Maps/Town02', '/Game/Carla/Maps/Town02_Opt', '/Game/Carla/Maps/Town03', '/Game/Carla/Maps/Town03_Opt', '/Game/Carla/Maps/Town04', '/Game/Carla/Maps/Town04_Opt', '/Game/Carla/Maps/Town05', '/Game/Carla/Maps/Town05_Opt', '/Game/Carla/Maps/Town10HD', '/Game/Carla/Maps/Town10HD_Opt']


In [17]:
world = client.load_world('Town01_Opt')

In [6]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [26]:
import torch

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")

if (torch.cuda.is_available()):
    print(f"CUDA version: {torch.version.cuda}")

    # Storing ID of current CUDA device
    cuda_id = torch.cuda.current_device()
    print(f"ID of current CUDA device:{torch.cuda.current_device()}")

    print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? False


In [16]:
for sign in world.get_actors().filter('traffic.speed_limit.90'):
    sign.destroy()

In [11]:
dir(world.get_actors().filter('traffic.speed_limit.90')[1])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'actor_state',
 'add_angular_impulse',
 'add_force',
 'add_impulse',
 'add_torque',
 'attributes',
 'bounding_box',
 'destroy',
 'disable_constant_velocity',
 'enable_constant_velocity',
 'get_acceleration',
 'get_angular_velocity',
 'get_location',
 'get_transform',
 'get_velocity',
 'get_world',
 'id',
 'is_active',
 'is_alive',
 'is_dormant',
 'parent',
 'semantic_tags',
 'set_enable_gravity',
 'set_location',
 'set_simulate_physics',
 'set_target_angular_velocity',
 'set_target_velocity',
 'set_transform',
 'trigger_volume',
 'type_id']

# Automatic Vehicle View

In [8]:
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [15]:
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation)
# we need x = 0 for sign detection
spectator.set_transform(transform)

In [7]:
for i in range(30):
    vehicle_bp = random.choice(bp_lib.filter('vehicle'))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [12]:
vehicle.set_autopilot(True)

In [29]:
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

In [84]:
## If want to destroy
for v in world.get_actors().filter('*vehicle*'):
    v.destroy()

# Walkers

In [11]:
for i in range(30):
    walker_bp = random.choice(bp_lib.filter('walker'))
    world.try_spawn_actor(walker_bp, random.choice(spawn_points))

# Camera stuff

In [12]:
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(x=0,z=2.5))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [13]:
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))


In [14]:
camera.stop()

# Testings

In [30]:
print(dir(carla))

['Actor', 'ActorAttribute', 'ActorAttributeType', 'ActorBlueprint', 'ActorList', 'ActorSnapshot', 'ActorState', 'AttachmentType', 'BlueprintLibrary', 'BoundingBox', 'CityObjectLabel', 'Client', 'ClientSideSensor', 'CollisionEvent', 'Color', 'ColorConverter', 'DVSEvent', 'DVSEventArray', 'DebugHelper', 'EnvironmentObject', 'FakeImage', 'GearPhysicsControl', 'GeoLocation', 'GnssMeasurement', 'IMUMeasurement', 'Image', 'Junction', 'LabelledPoint', 'Landmark', 'LandmarkOrientation', 'LandmarkType', 'LaneChange', 'LaneInvasionEvent', 'LaneInvasionSensor', 'LaneMarking', 'LaneMarkingColor', 'LaneMarkingType', 'LaneType', 'LidarDetection', 'LidarMeasurement', 'Light', 'LightGroup', 'LightManager', 'LightState', 'Location', 'Map', 'MapLayer', 'ObstacleDetectionEvent', 'OpendriveGenerationParameters', 'OpticalFlowImage', 'OpticalFlowPixel', 'Osm2Odr', 'Osm2OdrSettings', 'RadarDetection', 'RadarMeasurement', 'Rotation', 'SemanticLidarDetection', 'SemanticLidarMeasurement', 'Sensor', 'SensorData'

In [31]:
print(dir(bp_lib.find("vehicle.nissan.patrol_2021")))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_attribute', 'has_attribute', 'has_tag', 'id', 'match_tags', 'set_attribute', 'tags']


In [36]:
vec2 = world.get_actors()

In [37]:
print(vec2)

[Actor(id=1, type=spectator), Actor(id=2, type=traffic.stop), Actor(id=3, type=traffic.stop), Actor(id=4, type=traffic.stop), Actor(id=5, type=traffic.yield), Actor(id=6, type=traffic.unknown), Actor(id=7, type=traffic.unknown), Actor(id=72, type=vehicle.mini.cooper_s_2021), Actor(id=8, type=traffic.unknown), Actor(id=73, type=sensor.other.collision), Actor(id=9, type=traffic.traffic_light), Actor(id=74, type=sensor.other.lane_invasion), Actor(id=10, type=traffic.traffic_light), Actor(id=75, type=sensor.other.gnss), Actor(id=11, type=traffic.traffic_light), Actor(id=76, type=sensor.other.imu), Actor(id=12, type=traffic.traffic_light), Actor(id=77, type=sensor.camera.rgb), Actor(id=13, type=traffic.traffic_light), Actor(id=14, type=traffic.traffic_light), Actor(id=15, type=traffic.traffic_light), Actor(id=16, type=traffic.traffic_light), Actor(id=17, type=traffic.traffic_light), Actor(id=18, type=traffic.traffic_light), Actor(id=19, type=traffic.traffic_light), Actor(id=20, type=traffic

In [23]:
print(dir(vec2))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'filter', 'find']


In [38]:
print(dir(vec2[8]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'actor_state', 'add_angular_impulse', 'add_force', 'add_impulse', 'add_torque', 'attributes', 'bounding_box', 'destroy', 'disable_constant_velocity', 'enable_constant_velocity', 'get_acceleration', 'get_angular_velocity', 'get_location', 'get_transform', 'get_velocity', 'get_world', 'id', 'is_active', 'is_alive', 'is_dormant', 'parent', 'semantic_tags', 'set_enable_gravity', 'set_location', 'set_simulate_physics', 'set_target_angular_velocity', 'set_target_velocity', 'set_transform', 'trigger_volume', 'type_id']


In [63]:
v = vec2[7].get_velocity()
v_magnitude = 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2);
print(v,v_magnitude)

Vector3D(x=-16.609076, y=-0.116656, z=-0.048151) 59.79439805013903


In [46]:
print(vec2[7].get_velocity())

Vector3D(x=1.972452, y=4.576678, z=0.002487)


In [191]:
vec2[46].use_carsim_road(True)

In [211]:
vec2[46].set_target_velocity(carla.Vector3D(6,0,0))

In [164]:
vec2[46].set_enable_gravity(False)

In [73]:
print(world.get_actors())

[Actor(id=191, type=spectator), Actor(id=192, type=traffic.traffic_light), Actor(id=193, type=traffic.traffic_light), Actor(id=194, type=traffic.traffic_light), Actor(id=195, type=traffic.traffic_light), Actor(id=196, type=traffic.traffic_light), Actor(id=197, type=traffic.traffic_light), Actor(id=198, type=traffic.traffic_light), Actor(id=199, type=traffic.traffic_light), Actor(id=200, type=traffic.traffic_light), Actor(id=201, type=traffic.traffic_light), Actor(id=202, type=traffic.traffic_light), Actor(id=203, type=traffic.traffic_light), Actor(id=204, type=traffic.traffic_light), Actor(id=205, type=traffic.traffic_light), Actor(id=206, type=traffic.traffic_light), Actor(id=207, type=traffic.traffic_light), Actor(id=208, type=traffic.traffic_light), Actor(id=209, type=traffic.traffic_light), Actor(id=210, type=traffic.traffic_light), Actor(id=211, type=traffic.traffic_light), Actor(id=212, type=traffic.traffic_light), Actor(id=213, type=traffic.traffic_light), Actor(id=214, type=tra

In [78]:
print(world.get_actors().filter('traffic.speed_limit.*'))

[Actor(id=216, type=traffic.speed_limit.90), Actor(id=217, type=traffic.speed_limit.90), Actor(id=218, type=traffic.speed_limit.60), Actor(id=219, type=traffic.speed_limit.60), Actor(id=220, type=traffic.speed_limit.60), Actor(id=221, type=traffic.speed_limit.60), Actor(id=222, type=traffic.speed_limit.60), Actor(id=223, type=traffic.speed_limit.60), Actor(id=224, type=traffic.speed_limit.60), Actor(id=225, type=traffic.speed_limit.30), Actor(id=226, type=traffic.speed_limit.30), Actor(id=227, type=traffic.speed_limit.30), Actor(id=228, type=traffic.speed_limit.30), Actor(id=229, type=traffic.speed_limit.30), Actor(id=230, type=traffic.speed_limit.30), Actor(id=231, type=traffic.speed_limit.30), Actor(id=232, type=traffic.speed_limit.30), Actor(id=233, type=traffic.speed_limit.30), Actor(id=234, type=traffic.speed_limit.30), Actor(id=235, type=traffic.speed_limit.30), Actor(id=236, type=traffic.speed_limit.30)]


In [80]:
speed_limits = world.get_actors().filter('traffic.speed_limit.*')

In [81]:
len(speed_limits)

21

In [ ]:
# v = world.player.get_velocity()
# v_magnitude = 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2);
# # trying to set speed to 1
# world.player.set_target_velocity((1/v_magnitude)*v)
# world.hud.notification("Override Speed: "+(1/v_magnitude)*v) 